# Họ tên: Nguyễn Vũ Hải
# Lớp: CNTT K18 CLC
# Mã SV: DTC195480201CLC0006

# Bài tập mẫu


**Viết ứng dụng Wordcount dùng khung MapReduce:** Đếm số lần xuất hiện của mỗi từ (word) trong tệp dữ liệu đầu vào

Gợi ý:

- Xây dựng Mapper và Reducer riêng lẻ

- Với hỗ trợ của gói `Hadoop streaming` (Link:https://hadoop.apache.org/docs/r1.2.1/streaming.html#How+Streaming+Works),  Mapper và Reducer là các tệp thực thi đọc đầu vào từ luồng stdin (theo từng dòng) và ghi kết quả đầu ra cho stdout. 

- Mapper: Đọc từng dòng từ stdin và tách các từ; sau đó, mỗi từ được gán cho số đếm = 1. Lưu dữ liệu đầu ra dưới dạng <key,value> pair

- Map Output cần được sắp xếp (sort) trước khi đưa vào Reducer.

- Reducer: Khi tác vụ Reduce chạy, nó sẽ chuyển đổi các cặp khóa/giá trị đầu vào của nó thành các dòng và cung cấp các dòng này cho stdin của tiến trình (process). Trong quá trình chạy, Reducer thu thập các kết quả hướng dòng từ stdout của tiến trình, chuyển đổi từng dòng thành một <key,value> pair.


**Tạo Mapper**

In [6]:
%%writefile word_count_mapper2.py

import sys 


for line in sys.stdin:

    # remove leading and trailing whitespace 
    line = line.strip() 
    # split the line into words 
    words = line.split() 
    # increase counters 
    for word in words: 
        print('%s\t%s' % (word, 1))


Overwriting word_count_mapper2.py


**Chạy thử kiểm tra Mapper với bash script (sử dụng Cell magic %%bash)**

In [1]:
%%cmd
type lyric.txt | python word_count_mapper2.py  

Microsoft Windows [Version 10.0.19044.2728]
(c) Microsoft Corporation. All rights reserved.

(NLP) F:\github\BigDataWork\Work>type lyric.txt | python word_count_mapper2.py  
Ever	1
wonder	1
'bout	1
what	1
he's	1
doin'?	1
How	1
it	1
all	1
turned	1
to	1
lies?	1
Sometimes	1
I	1
think	1
that	1
it's	1
better	1
To	1
never	1
ask	1
why	1
Where	1
there	1
is	1
desire,	1
there	1
is	1
gonna	1
be	1
a	1
flame	1
Where	1
there	1
is	1
a	1
flame,	1
someone's	1
bound	1
to	1
get	1
burned	1
But	1
just	1
because	1
it	1
burns	1
doesn't	1
mean	1
you're	1
gonna	1
die	1
You've	1
gotta	1
get	1
up	1
and	1
try,	1
try,	1
try	1
Gotta	1
get	1
up	1
and	1
try,	1
try,	1
try	1
Gotta	1
get	1
up	1
and	1
try,	1
try,	1
try	1
Funny	1
how	1
the	1
heart	1
can	1
be	1
deceiving	1
More	1
than	1
just	1
a	1
couple	1
times	1
Why	1
do	1
we	1
fall	1
in	1
love	1
so	1
easy	1
Even	1
when	1
it's	1
not	1
right?	1
Where	1
there	1
is	1
desire,	1
there	1
is	1
gonna	1
be	1
a	1
flame	1
Where	1
there	1
is	1
a	1
flame,	1
someone's	1
bound	1
to	1
g

**Tạo Reducer**

In [15]:
%%writefile word_count_reducer2.py

import sys
current_word = None
current_count = 0
word = None
for line in sys.stdin:
    # remove leading and trailing whitespaces
    line = line.strip()
    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print ('%s\t%s' % (current_word, current_count))
        current_count = count
        current_word = word
if current_word == word:
    print('%s\t%s' % (current_word, current_count))

Overwriting word_count_reducer2.py


# Thực thi ứng dụng trong Python:

Chạy 'MapReduce job' dùng chuỗi câu lệnh bash script:

Streaming file -> Mapping -> Sorting -> Reducing

In [4]:
%%cmd
type lyric.txt | python word_count_mapper2.py   | sort | python word_count_reducer2.py

Microsoft Windows [Version 10.0.19044.2728]
(c) Microsoft Corporation. All rights reserved.

(NLP) F:\github\BigDataWork\Work>type lyric.txt | python word_count_mapper2.py   | sort | python word_count_reducer2.py
'bout	1
a	7
all	1
and	14
And	1
and	1
Are	1
are	1
ask	1
be	5
because	3
better	1
bound	3
burned	3
burns	3
But	3
by,	2
by?	2
can	1
couple	1
cry?	1
deceiving	1
desire,	3
die	3
do	1
does	1
doesn't	3
doin'?	1
doing	2
easy	1
Even	1
Ever	2
fall	1
flame	3
flame,	3
Funny	1
get	18
getting	2
gonna	6
gotta	2
Gotta	5
gotta	5
Gotta	3
he's	1
heart	1
How	1
how	1
I	1
in	1
is	9
it's	2
it	6
just	6
lies?	1
love	1
make	1
me,	1
mean	3
might	1
More	1
never	1
not	1
out	1
right?	1
ruined	1
so	1
someone's	3
Sometimes	1
Tell	1
than	1
that	2
the	1
there	10
think	1
times	1
to	4
To	1
try	15
try,	30
turned	1
up	15
wanna	1
we	1
what	2
When	1
when	1
Where	6
Why	1
why	1
wonder	1
worry	1
you're	5
You've	7
you	3

(NLP) F:\github\BigDataWork\Work>

# Chạy Python MapReduce trong Hadoop 

**Để chạy trong Hadoop 1 ứng dụng MapReduce viết bằng Python, sử dụng gói Hadoop Streaming**.

Ngoài ra, các thư mục lưu dữ liệu đầu vào, lưu kết quả đầu ra cần được xác lập.

%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=1
hdfs dfs cp -copyFromLocal "lyric.txt" /user/input-dir

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

$HADOOP_HOME/bin/hadoop  jar $HADOOP_HOME/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files word_count_mapper2.py,word_count_reducer2.py \
    -mapper "python word_count_mapper2.py" \
    -combiner "python word_count_reducer2.py" \
    -reducer "python word_count_reducer2.py" \
    -input /user/input-dir/\
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

# Bài tập thực hành

Bài 1. Viết ứng dụng MapReduce để sắp xếp từ theo số lần xuất hiện giảm dần.


In [18]:
%%writefile word_count_sort_reducer2.py

import sys
current_word = None
current_count = 0
word = None
dictNum = {}
for line in sys.stdin:
    # remove leading and trailing whitespaces
    line = line.strip()
    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue
    if current_word == word:
        current_count += count
    else:
        try:    
            if current_word:
                dictNum[word] += count
        except:
                dictNum[word] = count
        current_count = count
        current_word = word
FinalDict = sorted(dictNum.items(), key=lambda x:x[1], reverse=True)
converted_dict = dict(FinalDict)
for i in converted_dict:
    print(f"{i}: {converted_dict[i]}")

Overwriting word_count_sort_reducer2.py


In [21]:
%%cmd
type lyric.txt | python word_count_mapper2.py   | sort | python word_count_reducer2.py | python word_count_sort_reducer2.py

Microsoft Windows [Version 10.0.19044.2728]
(c) Microsoft Corporation. All rights reserved.

(NLP) F:\github\BigDataWork\Work>type lyric.txt | python word_count_mapper2.py   | sort | python word_count_reducer2.py | python word_count_sort_reducer2.py
try,: 30
get: 18
and: 15
try: 15
up: 15
there: 10
is: 9
Gotta: 8
a: 7
gotta: 7
You've: 7
gonna: 6
it: 6
just: 6
Where: 6
be: 5
you're: 5
to: 4
because: 3
bound: 3
burned: 3
burns: 3
But: 3
desire,: 3
die: 3
doesn't: 3
flame: 3
flame,: 3
mean: 3
someone's: 3
you: 3
by,: 2
by?: 2
doing: 2
Ever: 2
getting: 2
it's: 2
that: 2
what: 2
all: 1
And: 1
Are: 1
are: 1
ask: 1
better: 1
can: 1
couple: 1
cry?: 1
deceiving: 1
do: 1
does: 1
doin'?: 1
easy: 1
Even: 1
fall: 1
Funny: 1
he's: 1
heart: 1
How: 1
how: 1
I: 1
in: 1
lies?: 1
love: 1
make: 1
me,: 1
might: 1
More: 1
never: 1
not: 1
out: 1
right?: 1
ruined: 1
so: 1
Sometimes: 1
Tell: 1
than: 1
the: 1
think: 1
times: 1
To: 1
turned: 1
wanna: 1
we: 1
When: 1
when: 1
Why: 1
why: 1
wonder: 1
worry: 1

(NLP

Bài 2. Viết ứng dụng MapReduce grep để tìm các dòng xuất hiện một chuỗi mẫu (pattern) của tệp đầu vào.

In [60]:
%%writefile word_grep_mapper2.py

import sys
pattern = sys.argv[1]

# Đọc dữ liệu đầu vào từ stdin và kiểm tra các dòng có chứa chuỗi mẫu
for line in sys.stdin:
    if pattern in line:
        # Ghi ra cặp key-value với key là chuỗi mẫu và value là dòng chứa chuỗi mẫu
        print('{0}\t{1}'.format(pattern, line.strip()))

Overwriting word_grep_mapper2.py


In [91]:
%%writefile word_grep_reducer2.py

import sys

current_word = None
current_count = 0
word = None

# Danh sách các dòng chứa chuỗi mẫu
lines = set()
# Đọc các cặp key-value từ stdin
for line in sys.stdin:
    # Lấy key và value từ cặp key-value
    key, value = line.strip().split('\t', 1)
    lines.add(value)
    
for i in lines:
    print(i)

Overwriting word_grep_reducer2.py


In [92]:
%%cmd
type lyric.txt | python word_grep_mapper2.py "try" | sort | python word_grep_reducer2.py

Microsoft Windows [Version 10.0.19044.2728]
(c) Microsoft Corporation. All rights reserved.

(NLP) F:\github\BigDataWork\Work>type lyric.txt | python word_grep_mapper2.py "try" | sort | python word_grep_reducer2.py
Gotta get up and try, try, try
You've gotta get up and try, try, try

(NLP) F:\github\BigDataWork\Work>